In [1]:
import cv2
import numpy as np

In [2]:
def ellipse_detection(img):
  
  blurred_img = cv2.medianBlur(img,5)      

  cv2.imshow('gray', blurred_img)
  cv2.waitKey(0)

  circles = cv2.HoughCircles(blurred_img,cv2.HOUGH_GRADIENT,.5, 30,
                              param1=130,param2=70,minRadius=3)
  center = []

  if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
      center.append((i[0],i[1]))

  return center

In [3]:
def get_contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 20)

    edged = cv2.Canny(blur,40,100,L2gradient=True)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilated = cv2.morphologyEx(edged, cv2.MORPH_DILATE, structuringElement)

    contours, _ = cv2.findContours(dilated,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [4]:
def get_contour_center(cnt):
    
    M = cv2.moments(cnt)
    cX = int(M['m10'] / M['m00'])
    cY = int(M['m01'] / M['m00'])

    return (cX,cY)

In [5]:
def get_k_nearest(poly, point, k=4):
    (x,y) = point
    return sorted(poly, key=lambda pt: ((pt[0][0] - x)**2 + (pt[0][1] - y)**2))[:k]

In [6]:
def cross_detection(image):

    contours = get_contours(image)

    cross_candidates = []

    for cnt in contours:

        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.01*perimeter, True)

        hull = cv2.convexHull(cnt, returnPoints=False)
        defects = cv2.convexityDefects(cnt, hull)

        if defects is None:
            continue
        
        if (8 <= len(approx) <= 14) and (cv2.isContourConvex(cnt) == False):
            
            center = get_contour_center(cnt)
            four_nearest = get_k_nearest(approx, center)

            cross_candidates.append((center, four_nearest))

    return cross_candidates


In [7]:
def landing_pad_detection(image):
    
    ellipse_candidates = ellipse_detection(image)
    cross_candidates = cross_detection(image)

    min_dist = 1e7
    points = []

    for ellipse_center in ellipse_candidates:
        for (cross_center, points_) in cross_candidates:
            dist = (ellipse_center[0] - cross_center[0])**2 + (ellipse_center[1] - cross_center[1])**2
            if dist < min_dist:
                min_dist = dist
                points = points_

    if len(points) == 0:
        return False, []

    return True, points

In [8]:
image = cv2.imread('image/landing_pad_1.png')
final_image = cv2.imread('image/landing_pad_1.png')


ret, points = landing_pad_detection(image)

if ret:
    for pt in points:
        cv2.circle(final_image, (pt[0][0], pt[0][1]), 7, (255, 255, 255), -1)

cv2.imshow('initial_image', image)
cv2.imshow('final_image', final_image)
cv2.waitKey(0)
